In [1]:
import pandas as pd

In [2]:
# Defining a dictionary that contains all the datasets
files = {
    "Joint_Malnutrition_Estimates": "C:\\Users\\hp\\Downloads\\Child_Malnutrition_Analysis\\JME_Country_Estimates_May_2023.xlsx",
    "FAOSTAT_Agriculture": "C:\\Users\\hp\\Downloads\\Child_Malnutrition_Analysis\\FAOSTAT_Agriculture.csv",
    "FAOSTAT_Food_Insecurity": "C:\\Users\\hp\\Downloads\\Child_Malnutrition_Analysis\\FAOSTAT_Food_Insecurity.csv",
    "World_Bank_CPI": "C:\\Users\\hp\\Downloads\\Child_Malnutrition_Analysis\\World_Bank_Consumer_Price_Index.csv",
    "World_Bank_Fertility_Rate" :"C:\\Users\\hp\\Downloads\\Child_Malnutrition_Analysis\\World_Bank_Fertility_Rate.csv",
    "World_Bank_GDP_Per_Capita": "C:\\Users\\hp\\Downloads\\Child_Malnutrition_Analysis\\World_Bank_GDP_Per_Capita.csv",
    "World_Bank_Population_Total": "C:\\Users\\hp\\Downloads\\Child_Malnutrition_Analysis\\World_Bank_Population_Total.csv",
    "World_Bank_Population_Growth": "C:\\Users\\hp\\Downloads\\Child_Malnutrition_Analysis\\World_Bank_Population_Growth.csv",
    "World_Bank_Poverty_at_3.65": "C:\\Users\\hp\\Downloads\\Child_Malnutrition_Analysis\\World_Bank_Poverty_at_3.65$.csv",
    "World_Bank_Poverty_at_Societal_Line": "C:\\Users\\hp\\Downloads\\Child_Malnutrition_Analysis\\World_Bank_Poverty_at_Societal_Line.csv"
}

In [3]:
# Loading the datasets and skipping unnecessary rows
agriculture_data = pd.read_csv(files["FAOSTAT_Agriculture"])
food_insecurity = pd.read_csv(files["FAOSTAT_Food_Insecurity"])
gdp_data = pd.read_csv(files["World_Bank_GDP_Per_Capita"], skiprows=4)
cpi_data = pd.read_csv(files["World_Bank_CPI"], skiprows=4)
fertility_rate = pd.read_csv(files["World_Bank_Fertility_Rate"], skiprows=4)
population_total = pd.read_csv(files["World_Bank_Population_Total"], skiprows=4)
population_growth = pd.read_csv(files["World_Bank_Population_Growth"], skiprows=4)
poverty_365 = pd.read_csv(files["World_Bank_Poverty_at_3.65"], skiprows=4)
poverty_societal = pd.read_csv(files["World_Bank_Poverty_at_Societal_Line"], skiprows=4)

# Loading the child malnutrition data Using the sheets that provide modeled estimates
jme_sheets = ['Stunting Proportion (Model)', 'Overweight Proportion (Model)',
              'Stunting Numb Affected(Model)', 'Overweight Numb Affected(Model)']
child_malnutrition = pd.read_excel(files["Joint_Malnutrition_Estimates"], sheet_name= jme_sheets)

In [4]:
# Defining countries in the MENA region
mena_countries = [
    "Algeria", "Bahrain", "Djibouti", "Egypt", "Iran, Islamic Rep.", "Iraq", "Jordan",
    "Kuwait", "Lebanon", "Libya", "Morocco", "Oman", "Palestine", "Qatar",
    "Saudi Arabia", "Sudan", "Syria", "Tunisia", "United Arab Emirates", "Yemen"
]

In [5]:
# Standardizing Palestine in all datasets (Some datasets define 'West Bank and Gaza' as Palestine)
def renaming_Palestine(df, country_col):
    df[country_col] = df[country_col].replace(['West Bank and Gaza', 'State of Palestine'], 'Palestine')
    return df

In [6]:
# Standardizing Iran, Isalmic Rep. in all datasets
def renaming_Iran(df, contry_col):
    df[contry_col] = df[contry_col].replace('Iran (Islamic Republic of)', 'Iran, Islamic Rep.')
    return df

# Applying the function to relevant datasets
agriculture_data = renaming_Iran(agriculture_data, 'Area')
food_insecurity = renaming_Iran(food_insecurity, 'Area')

In [7]:
# Applying the function to relevant datasets (Datasets that has 'West Bank and Gaza' as a country)
gdp_data = renaming_Palestine(gdp_data, 'Country Name')
cpi_data = renaming_Palestine(cpi_data, 'Country Name')
fertility_rate = renaming_Palestine(fertility_rate, 'Country Name')
population_growth = renaming_Palestine(population_growth, 'Country Name')
population_total = renaming_Palestine(population_total, 'Country Name')
poverty_365 = renaming_Palestine(poverty_365, 'Country Name')
poverty_societal = renaming_Palestine(poverty_societal, 'Country Name')

In [8]:
# Iterating through the child malnutrition sheets and applying the renaming functions
for sheet_name, df in child_malnutrition.items():
    child_malnutrition[sheet_name] = renaming_Palestine(df, 'Country and areas')
    child_malnutrition[sheet_name] = renaming_Iran(child_malnutrition[sheet_name], 'Country and areas')

In [9]:
# Filtering the datasets to only include the MENA countries
def mena_filter(df, country_col):
    return df[df[country_col].isin(mena_countries)]

# Applying the function to relevant datasets
gdp_data = mena_filter(gdp_data, 'Country Name')
cpi_data = mena_filter(cpi_data, 'Country Name')
fertility_rate = mena_filter(fertility_rate, 'Country Name')
population_growth = mena_filter(population_growth, 'Country Name')
population_total = mena_filter(population_total, 'Country Name')
poverty_365 = mena_filter(poverty_365, 'Country Name')
poverty_societal = mena_filter(poverty_societal, 'Country Name')

In [10]:
# Iterating through the child malnutrition sheets and applying the filtering function
for sheet_name, df in child_malnutrition.items():
    child_malnutrition[sheet_name] = mena_filter(df, 'Country and areas')

In [11]:
# Checking if the filtering was successful
for i in child_malnutrition[sheet_name]['Country and areas'].unique():
    print(i)

Algeria
Bahrain
Djibouti
Egypt
Iran, Islamic Rep.
Iraq
Jordan
Kuwait
Lebanon
Libya
Morocco
Oman
Qatar
Saudi Arabia
Palestine
Sudan
Tunisia
United Arab Emirates
Yemen


In [12]:
# Converting JME data from wide format to long format for easier analysis
def wide_to_long(df, value_name):
    id_vars = ['ISO code', 'Country and areas']
    df_long = df.melt(id_vars = id_vars, var_name = 'Year', value_name = value_name)
    df_long['Year'] = pd.to_numeric(df_long['Year'], errors='coerce')
    return df_long

# Applying the function to the JME dataset sheets
stunting_model = wide_to_long(child_malnutrition['Stunting Proportion (Model)'], 'Stunting Proportion')
overweight_model = wide_to_long(child_malnutrition['Overweight Proportion (Model)'], 'Overweight Proportion')
stunting_affected_model = wide_to_long(child_malnutrition['Stunting Numb Affected(Model)'], 'Stunting Numb Affected')
overweight_affected_model = wide_to_long(child_malnutrition['Overweight Numb Affected(Model)'], 'Overweight Numb Affected')

In [20]:
# Merging the JME dataset sheets into one data frame
malnutrition_df = stunting_model.merge(overweight_model, on=['ISO code', 'Country and areas', 'Year'], how='outer')
malnutrition_df = malnutrition_df.merge(stunting_affected_model, on=['ISO code', 'Country and areas', 'Year'], how='outer')
malnutrition_df = malnutrition_df.merge(overweight_affected_model, on=['ISO code', 'Country and areas', 'Year'], how='outer')

# Renaming the columns 
malnutrition_df = malnutrition_df.rename(columns={'Country and areas': 'Country'})

In [23]:
# Renaming columns in all datasets for consistency
gdp_data = gdp_data.rename(columns={'Country Name': 'Country'})
cpi_data = cpi_data.rename(columns={'Country Name': 'Country'})
fertility_rate = fertility_rate.rename(columns={'Country Name': 'Country'})
population_growth = population_growth.rename(columns={'Country Name': 'Country'})
population_total = population_total.rename(columns={'Country Name': 'Country'})
poverty_365 = poverty_365.rename(columns={'Country Name': 'Country'})
poverty_societal = poverty_societal.rename(columns={'Country Name': 'Country'})
agriculture_data = agriculture_data.rename(columns={'Area': 'Country'})
food_insecurity = food_insecurity.rename(columns={'Area': 'Country'})